In [5]:
import cornac
from cornac.eval_methods import RatioSplit
from cornac.models import MF, PMF, BPR, Recommender
from cornac.metrics import MAE, RMSE, Precision, Recall, NDCG, AUC, MAP
from cornac.exception import ScoreException
import numpy as np
from sklearn.neural_network import MLPRegressor
from cornac.data import Dataset
from typing import List
import pandas as pd
class SimpleEnsemble(Recommender):
    def __init__(self, models:List[Recommender]=None, meta_learner=None,  *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.models : List[Recommender] = models
        self.meta_learner = meta_learner
    def fit(self, train_set:Dataset, val_set:Dataset=None):
        Recommender.fit(self, train_set, val_set)

        uir = np.array(train_set.uir_tuple).T
        df_uir:pd.DataFrame = pd.DataFrame(uir[:,[0,1]]).astype(int)
        df_uir=df_uir.groupby(0).aggregate(lambda x: list(map(int,x)))
        user_items = df_uir.to_dict()
        # print(df_uir)
        # print(df_uir.to_dict())
        
        # models_scores=[]
        for model in self.models:
            model.fit(train_set,val_set)
            for user, items in user_items.items():
                scores = model.score(user,items)
                
            # scores = model.score(uir[:,0].astype(int),uir[:,1].astype(int))
            models_scores.append(scores)
        # self.item_pop = np.ediff1d(train_set.csc_matrix.indptr)
        return self
    def score(self, user_idx, item_idx=None):
        if item_idx is None:
            return self.item_pop
        else:
            if self.train_set.is_unk_item(item_idx):
                raise ScoreException(
                    "Can't make score prediction for (user_id=%d, item_id=%d)"
                    % (user_idx, item_idx)
                )
            return self.item_pop[item_idx]

# load the built-in MovieLens 100K and split the data based on ratio
ml_100k = cornac.datasets.movielens.load_feedback()
rs = RatioSplit(data=ml_100k, test_size=0.2, rating_threshold=4.0, seed=123)

# initialize models, here we are comparing: Biased MF, PMF, and BPR
models = [
    SimpleEnsemble(models=[PMF(k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=123),MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123)],meta_learner=MLPRegressor(),name='SimpleEnsemble'),
    MF(k=10, max_iter=25, learning_rate=0.01, lambda_reg=0.02, use_bias=True, seed=123),
    # PMF(k=10, max_iter=100, learning_rate=0.001, lambda_reg=0.001, seed=123),
    # BPR(k=10, max_iter=200, learning_rate=0.001, lambda_reg=0.01, seed=123),
]

# define metrics to evaluate the models
metrics = [MAE(), RMSE(), Precision(k=10), Recall(k=10), NDCG(k=10), AUC(), MAP()]

# put it together in an experiment, voilà!
cornac.Experiment(eval_method=rs, models=models, metrics=metrics, user_based=True).run()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()